In [1]:
pip install requests



[notice] A new release of pip available: 22.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load Necessary Libraries

from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import sleep

In [3]:
# website url
url = "https://books.toscrape.com/"


In [4]:
products = []

def scrap_data(currentPage_url):
    books_data = requests.get(currentPage_url)

    #get Html elements
    html = BeautifulSoup(books_data.text,"html.parser")
    
    books = html.find_all('li', attrs={'class','col-xs-6 col-sm-4 col-md-3 col-lg-3'})
    
    for book in books:
        price = book.select('.price_color')[0].get_text()
        title = book.select('img')[0].get('alt')
        image = book.select('img')[0].get('src')
        availability = book.find('p', attrs={'class','instock availability'}).get_text()
        ratings = book.select('p')[0].get('class')[1]
        products.append({'title': title,'price': price, 'image':image, 'availability': availability, 'ratings': ratings})

    return products

In [5]:
#handling pagination

allBooks = []

def scrape_all_books(base_url,totalPages):
    for Pages in range(1, totalPages + 1 ):
        currentPage_url = f"{base_url}{Pages}.html"
        sleep(2)
        books = scrap_data(currentPage_url)
        allBooks.extend(books)
    return books

In [6]:
base_url="https://books.toscrape.com/catalogue/page-"
totalPages=50

all_books = scrape_all_books(base_url, totalPages)

# to store in csv file
df = pd.DataFrame(all_books)
df.to_csv("Books_S0up.csv",index=False)